# Checking for Null Values in Dataset

In [3]:
def checker (df):
    if (df.isnull().sum().sum())>0:
        print('Error. Data Contains Missing Values. Please Do Missing value treatment to do this step.')
        return 0
    return 1

# Dataset Input

In [4]:
def inputter():
    import pandas as pd
    fn=input('Enter File Name:')
    df=pd.read_csv(fn)
    return df

# Model Input Will be done from Model Library Added a list for reference

# K-fold Cross Validation

In [5]:
def cross_validation():
    from sklearn.model_selection import StratifiedKFold
    fold=int(input('Enter number of folds: (Enter 0 for default value)')) #Between 2-10
    if fold==0:
        fold=5
    shu=input('Shuffule? (Y/N):')
    if (shu=='Y'):
        shu=True
    else:
        shu=False
    cv=StratifiedKFold(n_splits=fold,shuffle=shu,random_state=True)
    return cv

# Converting Object type Data to Labels 

In [6]:
def data_types(column,data):
    X=data[column]
    objList = X.select_dtypes(include = "object").columns
    from sklearn.preprocessing import LabelEncoder
    le = LabelEncoder()
    #d={}   

    for feat in objList:
        X[feat] = le.fit_transform(X[feat].astype(str))
        le_name_mapping = dict(zip(le.classes_, le.transform(le.classes_)))
        #d[feat]=le_name_mapping
    return X

# Choose Target

In [7]:
def target_valid(data):
  import numpy as np
  valid_col=[]
  col=list(data.columns)
  for fet in col:
    x=len(np.unique(data[fet]))
    if (x<3):
      valid_col.append(fet)
  return valid_col

# Main function which will do the work

In [13]:
def main(data):
    from sklearn.model_selection import KFold
    from sklearn.preprocessing import StandardScaler
    from sklearn.linear_model import LogisticRegression
    from sklearn.model_selection import cross_val_score
    from sklearn.svm import SVC
    from sklearn.naive_bayes import GaussianNB
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.tree import DecisionTreeClassifier
    import xgboost as xgb
    import pandas as pd
    import statistics
    from scipy import interp
    import matplotlib.pyplot as plt
    from itertools import cycle
    from sklearn.metrics import roc_curve, auc
    import plotly.express as px
    import numpy as np
    import plotly.graph_objects as go
    cols=list(data.columns)
    df=data_types(cols,data)
    # create a scaler object
    std_scaler = StandardScaler()
    target_list=target_valid(data)
    print('CHOOSE THE APPROPRIATE TARGET COLUMN NAME FROM THE CHOICES:')
    print(target_list)
    target=input('Enter:')
    cols.remove(target)
    x=df.drop(target,axis=1)
    x=pd.DataFrame(std_scaler.fit_transform(x), columns=x.columns)
    y=df[target]
    x=x.to_numpy()
    models = []
    models.append(("LR",LogisticRegression()))
    models.append(("NB",GaussianNB()))
    models.append(("RF",RandomForestClassifier()))
    #models.append(("SVC",SVC(probability=True)))
    models.append(("Dtree",DecisionTreeClassifier()))
    models.append(("XGB",xgb.XGBClassifier()))
    #models.append(("KNN",KNeighborsClassifier()))
    kfold=cross_validation()
    names=['Logistic Regression','Gaussian Naive Bayes Classifier','Random Forest Classifier','Decision Tree Classifier','XGBoostClassifier']
    meanacc=[]
    maxacc=[]
    minacc=[]
    meanf1=[]
    maxf1=[]
    minf1=[]
    meanroc=[]
    maxroc=[]
    minroc=[]
    meanprec=[]
    maxprec=[]
    minprec=[]
    maxrec=[]
    meanrec=[]
    minrec=[]
    qw=0
    col=['red','yellow','blue','green','orange','purple','violet','black','pink','brown']
    for name,model in models:
      namy=names[qw]
      #kfold = KFold(n_splits=2)
      cv_result1 = cross_val_score(model,x,y, cv = kfold,scoring = "accuracy",n_jobs=-1)
      meanacc.append(round(statistics.mean(cv_result1),4))
      maxacc.append(round(cv_result1.max(),4))
      minacc.append(round(cv_result1.min(),4))
      cv_result2= cross_val_score(model,x,y, cv = kfold,scoring = "f1",n_jobs=-1)
      meanf1.append(round(statistics.mean(cv_result2),4))
      maxf1.append(round(cv_result2.max(),4))
      minf1.append(round(cv_result2.min(),4))
      cv_result3 = cross_val_score(model,x,y, cv = kfold,scoring = "roc_auc",n_jobs=-1)
      meanroc.append(round(statistics.mean(cv_result3),4))
      maxroc.append(round(cv_result3.max(),4))
      minroc.append(round(cv_result3.min(),4))
      cv_result4 = cross_val_score(model,x,y, cv = kfold,scoring = "average_precision",n_jobs=-1)
      meanprec.append(round(statistics.mean(cv_result4),4))
      maxprec.append(round(cv_result4.max(),4))
      minprec.append(round(cv_result4.min(),4))
      cv_result5 = cross_val_score(model,x,y, cv = kfold,scoring = "recall",n_jobs=-1)
      meanrec.append(round(statistics.mean(cv_result5),4))
      maxrec.append(round(cv_result5.max(),4))
      minrec.append(round(cv_result5.min(),4))
      fig = go.Figure()
      i=1
      fig.add_trace(go.Scatter(x=[0,1], y=[0,1],
                    mode='lines',
                    name='No Skill',
                    line=dict(color='black', width=0.2,dash = 'dash')))
      for (train, test) in kfold.split(x, y):
        probas_ = model.fit(x[train], y[train]).predict_proba(x[test])
        # Compute ROC curve and area the curve
        fpr, tpr, thresholds = roc_curve(y[test], probas_[:, 1])
        roc_auc_lr = auc(fpr, tpr)
        man='Fold '+str(i)+' AUC Score= '+str(round(roc_auc_lr,2))
        fig.add_trace(go.Scatter(x=fpr, y=tpr,
                    mode='lines',
                    name=man,
                    line=dict(color=col[i-1],width=1)))
        i+=1
      tit='ROC Curve for '+namy
      fig.update_layout(title=tit,
                        yaxis_title="<b> True Positive Rate </b>",
                        xaxis_title="<b> False Positive Rate </b>",
                        font=dict(
                            family="Courier New, monospace",
                            size=13,
                            color="#7f7f7f")
                        )
      fig.update_layout({'plot_bgcolor': 'rgba(0,0,0,0)', 'paper_bgcolor': 'rgba(0,0,0,0)'})
      fig.show()
      qw=qw+1
      #colors = cycle(['cyan', 'indigo', 'seagreen', 'yellow', 'blue', 'darkorange'])
      #lw = 2
      #i = 0
      #mean_tpr = 0.0
      #mean_fpr = np.linspace(0, 1, 100)
      #for (train, test), color in zip(kfold.split(x, y), colors):
        #probas_ = model.fit(x[train], y[train]).predict_proba(x[test])
        ## Compute ROC curve and area the curve
        #fpr, tpr, thresholds = roc_curve(y[test], probas_[:, 1])
        #mean_tpr += interp(mean_fpr, fpr, tpr)
        #mean_tpr[0] = 0.0
        #roc_auc = auc(fpr, tpr)
        #plt.plot(fpr, tpr, lw=lw, color=color,
         #        label='ROC fold %d (area = %0.2f)' % (i, roc_auc))
        #i += 1
      #plt.plot([0, 1], [0, 1], linestyle='--', lw=lw, color='k',
      #          label='Luck')
      #mean_tpr /= kfold.get_n_splits(x, y)
      #mean_tpr[-1] = 1.0
      #mean_auc = auc(mean_fpr, mean_tpr)
      #plt.plot(mean_fpr, mean_tpr, color='g', linestyle='--',
       #         label='Mean ROC (area = %0.2f)' % mean_auc, lw=lw)
      #plt.xlim([-0.05, 1.05])
      #plt.ylim([-0.05, 1.05])
      #plt.xlabel('False Positive Rate')
      #plt.ylabel('True Positive Rate')
      #plt.title('Receiver operating characteristic example')
      #plt.legend(loc="lower right")
      #plt.show()
    table=pd.DataFrame()
    table['Model Name']=names
    table['Minimum Accuracy Score']=minacc
    table['Mean Accuracy Score']=meanacc
    table['Maximum Accuracy Score']=maxacc
    table['Minimum f1 Score']=minf1
    table['Mean f1 Score']=meanf1
    table['Maximum f1 Score']=maxf1
    table['Minimum AUCROC Score']=minroc
    table['Mean AUCROC Score']=meanroc
    table['Maximum AUCROC Score']=maxroc
    table['Minimum Precision Score']=minprec
    table['Mean Precision Score']=meanprec
    table['Maximum Precision Score']=maxprec
    table['Minimum Recall Score']=minrec
    table['Mean Recall Score']=meanrec
    table['Maximum Recall Score']=maxrec
    display(table)

In [14]:
import pandas as pd
df=pd.read_csv('/content/Churn_Modelling.csv')
main(df)

CHOOSE THE APPROPRIATE TARGET COLUMN NAME FROM THE CHOICES:
['Gender', 'HasCrCard', 'IsActiveMember', 'Exited']
Enter:Exited
Enter number of folds: (Enter 0 for default value)10
Shuffule? (Y/N):Y


,Model Name,Minimum Accuracy Score,Mean Accuracy Score,Maximum Accuracy Score,Minimum f1 Score,Mean f1 Score,Maximum f1 Score,Minimum AUCROC Score,Mean AUCROC Score,Maximum AUCROC Score,Minimum Precision Score,Mean Precision Score,Maximum Precision Score,Minimum Recall Score,Mean Recall Score,Maximum Recall Score
0,Logistic Regression,0.795,0.8079,0.820,0.1667,0.2688,0.3453,0.7311,0.7534,0.7787,0.4002,0.4518,0.5087,0.0980,0.1753,0.2414
1,Gaussian Naive Bayes Classifier,0.821,0.8290,0.839,0.3095,0.3655,0.4145,0.7842,0.8049,0.8243,0.5254,0.5763,0.6314,0.1912,0.2430,0.2857
2,Random Forest Classifier,0.836,0.8591,0.872,0.4651,0.5518,0.5944,0.8251,0.8488,0.8786,0.6183,0.6726,0.7125,0.3480,0.4291,0.4657
3,Decision Tree Classifier,0.770,0.7830,0.801,0.4421,0.4835,0.5381,0.6614,0.6841,0.7153,0.3021,0.3407,0.3730,0.4020,0.5017,0.5441
4,XGBoostClassifier,0.840,0.8627,0.877,0.4737,0.5668,0.6261,0.8416,0.8628,0.8901,0.6328,0.6981,0.7514,0.3529,0.4423,0.5049
